| ![nsdf](https://www.sci.utah.edu/~pascucci/public/NSDF-smaller.PNG)  | [National Science Data Fabric](https://nationalsciencedatafabric.org/) [Jupyter notebook](https://jupyter.org/) <br> created by  [Valerio Pascucci](http://cedmav.com/) and  [Giorgio Scorzelli](https://www.sci.utah.edu/people/scrgiorgio.html)  |  
|---|:---:|


#  <center> This is a preview of the data:
![Visualization of Covid-19 cases](https://www.sci.utah.edu/~pascucci/public/DRP-preview.gif)

# Import OpenVisus and load dataset

In [1]:
import numpy as np
import panel as pn
import pandas as pd

import panel as pn
pn.extension('vtk') # VTK is needed for  VTKVolumePlot 

from openvisuspy import LoadDataset,ExecuteBoxQuery

url='http://atlantis.sci.utah.edu:80/mod_visus?dataset=fly_scan_time-s-midx/data&cached=1'
db=LoadDataset(url)
print(f"Loaded dataset \nfields={db.getFields()} \nlogic_box={db.getLogicBox()}")
W,H,D=db.getLogicBox()[1]
access=db.createAccess()

# VISUS_BACKEND=py
Loaded dataset 
fields=['data'] 
logic_box=[[0, 0, 0], [2000, 2000, 2160]]


# Show  data 

In [2]:
def MyApp(endh=21, width=700, height=1000):

    data=list(ExecuteBoxQuery(db, access=access, timestep=0, endh=endh, num_refinements=1))[0]['data']
    print(f"Got data shape={data.shape} dtype={data.dtype}")

     # generate a panel for numpy data
    volume = pn.panel(
        data, 
        sizing_mode='stretch_both', 
        height=200, 
        orientation_widget =True,
        display_slices=True,
        spacing = (1, 1, 1),
        controller_expanded=False)

    # other widgets, with callbacks
    experiment  = pn.widgets.IntSlider(name='Experiment', start=0, end=1, step=1, value=0 )
    @pn.depends(experiment)
    def experiment_callback(value):
        nonlocal volume
        data=list(ExecuteBoxQuery(db, access=access, endh=endh,timestep=value,num_refinements=1))[0]['data']
        volume.object = data
        return " "    
    
    slider_i = pn.widgets.IntSlider(name='i-slice', start=0, end=data.shape[0], value = data.shape[0]//2 )
    @pn.depends(slider_i)
    def slider_i_callback(value):
        nonlocal volume
        volume.slice_i = value 
        return " "    
    
    slider_j = pn.widgets.IntSlider(name='j-slice', start=0, end=data.shape[1], value = data.shape[1]//2 )
    @pn.depends(slider_j)
    def slider_j_callback(value):
        nonlocal volume
        volume.slice_j = value 
        return " "    
    
    slider_k = pn.widgets.IntSlider(name='k-slice', start=0, end=data.shape[2], value = data.shape[2]//2 )
    @pn.depends(slider_k)
    def slider_k_callback(value):
        nonlocal volume
        volume.slice_k = value 
        return " "    
    
    show_volume = pn.widgets.Checkbox(name='Show Volume',value=True)
    @pn.depends(show_volume)
    def show_volume_callback(value):
        nonlocal volume
        volume.display_volume = value 
        return " "    
    
    show_slices = pn.widgets.Checkbox(name='Show Slices',value=True)
    @pn.depends(show_slices)
    def show_slices_calback(value):
        nonlocal volume
        volume.display_slices = value 
        return " "    

    main_layout=pn.Column(
        experiment,
        slider_i,
        slider_j,
        slider_k,
        pn.Row(
            show_volume,
            show_slices
        ),
        volume, 
        experiment_callback,
        slider_i_callback,
        slider_j_callback,
        slider_k_callback,
        show_volume_callback,
        show_slices_calback,
        width = width,
        height = height,
        css_classes=['panel-widget-box'],
        sizing_mode='stretch_width',
        width_policy='max')

    return main_layout

MyApp()

Got data shape=(69, 126, 126) dtype=uint8


Column(css_classes=['panel-widget-box'], height=1000, sizing_mode='stretch_width', width=700, width_policy='max')
    [0] IntSlider(name='Experiment')
    [1] IntSlider(end=69, name='i-slice', value=34)
    [2] IntSlider(end=126, name='j-slice', value=63)
    [3] IntSlider(end=126, name='k-slice', value=63)
    [4] Row
        [0] Checkbox(name='Show Volume', value=True)
        [1] Checkbox(name='Show Slices', value=True)
    [5] VTKVolume(ndarray, controller_expanded=False, display_slices=True, height=200, orientation_widget=True, sizing_mode='stretch_both', slice_i=34, slice_j=63, slice_k=63)
    [6] ParamFunction(function, _pane=Markdown)
    [7] ParamFunction(function, _pane=Markdown)
    [8] ParamFunction(function, _pane=Markdown)
    [9] ParamFunction(function, _pane=Markdown)
    [10] ParamFunction(function, _pane=Markdown)
    [11] ParamFunction(function, _pane=Markdown)